# **Penting**

*   Pada Notebook ini, Anda hanya perlu mengerjakan code pada bagian **logic.py** saja. Anda tidak diwajibkan untuk mengubah atau menambahkan **app.py** yang digunakan untuk membangun interface Streamlit.
*   Namun, jika Anda memiliki preferensi lain atau ingin mengubah struktur code pada logic ataupun pada interface Streamlit, itu **DIPERSILAHKAN** saja, tetapi pastikan untuk memenuhi kriteria yang telah ditetapkan pada intruksi submission
*   Jika Anda tidak ingin mengubah apapun dan ingin mengikuti template, tugas Anda hanyalah melengkapi code yang rumpang pada bagian yang sudah ditandai "________" saja.



# **Prepare Dependencies**

In [1]:
!pip install -q pyngrok
!pip install -q streamlit
!pip install -q torch
!pip install -q diffusers
!pip install -q transformers
!pip install -q streamlit_drawable_canvas==0.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 125.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 138.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.2 MB/s eta 0:00:00


In [2]:
from pyngrok import ngrok
import subprocess

# **Streamlit**

## logic.py (**Basic**)

In [3]:
%%writefile logic.py
import torch
import gc
from diffusers import (
    StableDiffusionPipeline,
    StableDiffusionInpaintPipeline,
    EulerAncestralDiscreteScheduler,
    DPMSolverMultistepScheduler,
    DDIMScheduler
)
from PIL import Image, ImageFilter

# MODEL LOADER (JANGAN DIUBAH)
def load_models_cached():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Loading models to {device}")

    pipe_txt2img = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16
    ).to(device)

    pipe_inpaint = StableDiffusionInpaintPipeline.from_pretrained(
        "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16
    ).to(device)

    return pipe_txt2img, pipe_inpaint

# Ini mencegah error "Function not found" jika hanya mengerjakan Basic
def flush_memory(): pass
def set_scheduler(pipe, name): return pipe
def run_inpainting(pipe, img, mask, prompt, strength): return None
def prepare_outpainting(img, expand=128): return img, None

Writing logic.py


In [4]:
%%writefile -a logic.py

def generate_image(pipe, prompt, neg_prompt, seed, steps, cfg, num_images=1, scheduler_name="Euler A"):

    ### MULAI CODE ###

    # Setup Generator (Seed)
    generator = torch.Generator(device=pipe.device).manual_seed(seed)

    # Generate gambar standard
    image = pipe(
        prompt=prompt,
        negative_prompt=neg_prompt,
        num_inference_steps=steps,
        guidance_scale=cfg,
        generator=generator
    ).images[0]

    ### SELESAI CODE ###

    # Kembalikan dalam bentuk List agar kompatibel dengan UI (List isi 1 gambar)
    return [image]

Appending to logic.py


## logic.py (**Skilled**)

In [5]:
%%writefile -a logic.py

# Implementasi pembersihan RAM GPU
def flush_memory():

    ### MULAI CODE ###
    gc.collect()
    torch.cuda.empty_cache()
    ### SELESAI CODE ###

    print("Memory Flushed!")

# Ganti scheduler sesuai input
def set_scheduler(pipe, scheduler_name):

    ### MULAI CODE ###
    if scheduler_name == "Euler A":
        pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
    elif scheduler_name == "DPM++":
        pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    elif scheduler_name == "DDIM":
        pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
    ### SELESAI CODE ###

    return pipe

# Definisikan ulang fungsi generate_image dan tambahkan parameter untuk batch inference
def generate_image(pipe, prompt, neg_prompt, seed, steps, cfg, num_images=1, scheduler_name="Euler A"):

    ### MULAI CODE ###

    # Set Scheduler
    pipe = set_scheduler(pipe, scheduler_name)

    generator = torch.Generator(device=pipe.device).manual_seed(seed)

    # Generate Batch
    result = pipe(
        prompt=prompt,
        negative_prompt=neg_prompt,
        num_inference_steps=steps,
        guidance_scale=cfg,
        num_images_per_prompt=num_images,
        generator=generator
    ).images

    ### SELESAI CODE ###

    return result

Appending to logic.py


## logic.py (**Advanced**)

In [6]:
%%writefile -a logic.py

def run_inpainting(pipe, image, mask, prompt, strength):
    # Pastikan konversi RGB/L dan Resize Mask (Nearest)
    if image.mode != "RGB": image = image.convert("RGB")
    if mask.mode != "L": mask = mask.convert("L")

    # Resize Mask agar tajam
    if image.size != mask.size:
        mask = mask.resize(image.size, resample=Image.NEAREST)

    # Eksekusi Inpainting
    result = pipe(
        prompt=prompt,
        image=image,
        mask_image=mask,
        strength=strength
    ).images[0]

    return result

def prepare_outpainting(image, expand_pixels=128):

    ### MULAI CODE ###
    w, h = image.size
    new_w = w + (expand_pixels * 2)
    new_h = h + (expand_pixels * 2)

    # Safety: Resolusi kelipatan 8 agar model tidak error
    new_w -= (new_w % 8)
    new_h -= (new_h % 8)

    ### SELESAI CODE ###

    # Background Blur sebagai initial latent agar hasil blending natural
    bg = image.resize((new_w, new_h), resample=Image.BICUBIC)
    bg = bg.filter(ImageFilter.GaussianBlur(radius=50))

    canvas = bg.copy()
    paste_x = (new_w - w) // 2
    paste_y = (new_h - h) // 2
    canvas.paste(image, (paste_x, paste_y))

    # Masker (Putih = area baru untuk diisi, Hitam = area gambar asli untuk dijaga)
    mask = Image.new("L", (new_w, new_h), 255)
    inner_box = Image.new("L", (w, h), 0)

    ### MULAI CODE ###
    # Tempelkan kotak hitam di posisi gambar asli
    mask.paste(inner_box, (paste_x, paste_y))
    ### SELESAI CODE ###

    return canvas, mask

Appending to logic.py


## app.py
Anda **TIDAK perlu mengubah atau menambahkan** apapun pada file **app.py** ini, cukup **jalankan** saja.

In [7]:
%%writefile app.py
import streamlit as st
import torch
import numpy as np
from PIL import Image
from streamlit_drawable_canvas import st_canvas
import logic
from PIL import Image, ImageDraw, ImageOps, ImageFilter

# Config
st.set_page_config(page_title="StudioAI", layout="wide", page_icon="🎨")

# Load Models
@st.cache_resource
def get_models():
    return logic.load_models_cached()

try:
    pipe_txt2img, pipe_inpaint = get_models()
except Exception as e:
    st.error(f"Error loading models: {e}")
    st.stop()

st.title("🎨 StudioAI: Craeating Amazing Paint with Stable Diffusion")

with st.sidebar:
    st.header("⚙️ Parameters")
    # Basic
    steps = st.slider("Quality Steps", 15, 50, 30)
    cfg = st.slider("Creativity (CFG)", 1.0, 20.0, 7.5)
    seed = st.number_input("Seed Control", value=42)

    st.divider()

    # Skilled
    st.subheader("🚀 Advanced")
    scheduler_name = st.selectbox("Scheduler", ["Euler A", "DPM++", "DDIM"])
    num_images = st.slider("Batch Size (Jumlah Gambar)", 1, 4, 1)

    st.divider()
    if st.button("🧹 Flush RAM"):
        logic.flush_memory()
        st.toast("Memory Cleared!")

# Tab (fitur) yang tersedia
tab_gen, tab_edit = st.tabs(["✨ GENERATE", "🛠️ EDIT"])

# Tab Generate
with tab_gen:
    c1, c2 = st.columns([1, 1], gap="large")

    # Input
    with c1:
        st.subheader("Input Blueprint")
        with st.form(key="gen_form"):
            prompt = st.text_area("Prompt", "a cute robot in a futuristic city, 8k, masterpiece", height=150)
            neg_prompt = st.text_input("Negative Prompt", "blurry, bad anatomy, worst quality")

            submit_gen = st.form_submit_button("🚀 Initialize Generation", type="primary")

        if submit_gen:
            with st.spinner("Processing Image"):
                logic.flush_memory()

                generated_list = logic.generate_image(
                    pipe_txt2img, prompt, neg_prompt, seed, steps, cfg, num_images, scheduler_name
                )

                st.session_state['generated_images'] = generated_list

                # Ini mencegah error "List has no attribute .size" di Tab Edit.
                if generated_list:
                    st.session_state['current_image'] = generated_list[0]

            # Refresh halaman untuk update tampilan
            st.rerun()

    # Output
    with c2:
        st.subheader("Visual Output")

        if 'generated_images' in st.session_state:
            imgs = st.session_state['generated_images']

            # Batch image: ada lebih dari 1 gambar
            if len(imgs) > 1:
                cols = st.columns(2) # Grid 2 Kolom
                for idx, img in enumerate(imgs):
                    with cols[idx % 2]:
                        st.image(img, caption=f"Img {idx+1}", use_container_width=True)

                        # Tombol Pilih Gambar untuk diedit
                        if st.button(f"Select Img {idx+1}", key=f"sel_{idx}"):
                             st.session_state['current_image'] = img
                             st.toast(f"✅ Image {idx+1} Selected for Editing!")

            # Single Image
            elif len(imgs) == 1:
                st.image(imgs[0], caption="Result", use_container_width=True)

        else:
            st.info("👈 Masukkan prompt di panel kiri dan tekan Generate.")


# Tab Edit (Inpainting dan Outpainting)
with tab_edit:
    if 'current_image' in st.session_state:
        source_img = st.session_state['current_image']

        # Validasi tipe data
        if isinstance(source_img, list):
            st.warning("⚠️ Terdeteksi List Gambar. Mengambil gambar pertama secara otomatis.")
            source_img = source_img[0]
            st.session_state['current_image'] = source_img

        W, H = source_img.size

        # Pilihan Mode
        mode = st.radio("Select Mode:", ["Inpainting (Edit Objek)", "Outpainting (Zoom Out)"], horizontal=True)
        st.divider()

        # Mode Inpainting
        if mode == "Inpainting (Edit Objek)":
            col_tools, col_result = st.columns([1, 1], gap="large")

            # Logic Reset Canvas: Agar coretan hilang setelah generate
            if 'canvas_key' not in st.session_state:
                st.session_state['canvas_key'] = 0

            img_id = str(id(source_img))

            dynamic_key = f"canvas_{st.session_state['canvas_key']}_{img_id}"

            with col_tools:
                st.subheader("✍️ Draw Mask")
                st.caption("Warnai area yang ingin diubah.")

                # Widget Canvas
                canvas_result = st_canvas(
                    fill_color="rgba(255, 255, 255, 1.0)", # Warna Kuas Putih
                    stroke_width=20,
                    stroke_color="#FFFFFF",
                    background_image=source_img,
                    update_streamlit=True,
                    height=H, width=W,
                    drawing_mode="freedraw",
                    key=dynamic_key
                )

            with col_result:
                st.subheader("Settings")

                # Form Input
                with st.form("inpaint_input"):
                    edit_prompt = st.text_input("Prompt Baru (Ganti jadi apa?)", "a pair of sunglasses")
                    strength = st.slider("Strength (Seberapa kuat perubahannya?)", 0.1, 1.0, 0.85)
                    submit_inpaint = st.form_submit_button("⚡ Execute Inpainting", type="primary")

                # Logic Eksekusi
                if submit_inpaint:
                    if canvas_result.image_data is not None and np.max(canvas_result.image_data) > 0:

                        with st.spinner("Processing Inpainting..."):
                            logic.flush_memory()

                            # Proses Masker
                            # Ambil Alpha Channel
                            mask_data = canvas_result.image_data[:, :, 3]

                            # Ubah abu-abu jadi putih mutlak
                            mask_data[mask_data > 0] = 255
                            mask_image = Image.fromarray(mask_data.astype('uint8'), mode='L')

                            # Samakan ukuran mask dengan gambar asli (PENTING!)
                            if mask_image.size != source_img.size:
                                mask_image = mask_image.resize(source_img.size, resample=Image.NEAREST)

                            # Menebalkan atau mempertegas Masker
                            mask_image = mask_image.filter(ImageFilter.MaxFilter(15))

                            # Tampilkan Masker yang akan dilihat model
                            with st.expander("🕵️ Lihat Masker Final (Debug)"):
                                st.image(mask_image, caption="Masker Tajam (Tanpa Blur)", width=200)

                            try:
                                result_img = logic.run_inpainting(
                                    pipe_inpaint, source_img, mask_image, edit_prompt, strength
                                )

                                st.session_state['current_image'] = result_img
                                st.session_state['canvas_key'] = str(int(st.session_state.get('canvas_key', 0)) + 1)
                                st.success("Inpainting Selesai!")
                                st.rerun()

                            except Exception as e:
                                st.error(f"Error pada logic: {e}")
                    else:
                        st.warning("⚠️ Silakan coret gambar terlebih dahulu!")

        # Mode Outpainting
        elif mode == "Outpainting (Zoom Out)":
            c_out_1, c_out_2 = st.columns([1, 1], gap="large")

            with c_out_1:
                st.subheader("Original")
                st.image(source_img, caption="Gambar saat ini", use_container_width=True)

            with c_out_2:
                st.subheader("Expansion")
                with st.form("outpaint_input"):
                    st.info("Gambar akan diperluas 128px ke segala arah.")
                    out_prompt = st.text_input(
                        "Prompt Deskriptif (Jelaskan gambar UTUH)",
                        "wide angle view of [masukkan prompt awal], detailed background, 8k"
                    )
                    submit_outpaint = st.form_submit_button("🔍 Zoom Out (Expand)", type="primary")

                if submit_outpaint:
                    with st.spinner("Expanding Canvas..."):
                        logic.flush_memory()
                        try:
                            # Siapkan Canvas & Mask (Logic dari kode siswa di atas)
                            canvas_ready, mask_ready = logic.prepare_outpainting(source_img)

                            # Jalankan Inpainting pada area kosong
                            final_result = logic.run_inpainting(
                                pipe_inpaint, canvas_ready, mask_ready, out_prompt, 1.0
                            )
                            st.session_state['current_image'] = final_result
                            st.rerun()

                        except Exception as e:
                            st.error(f"Error pada logic Outpainting: {e}")
                            st.caption("Pastikan fungsi prepare_outpainting di logic.py sudah benar.")

    else:
        # Tampilan jika belum ada gambar sama sekali
        st.info("👈 Belum ada gambar. Silakan ke Tab 'GENERATE' dan buat gambar dulu.")

Writing app.py


# **Menggunakan *Ngrok* Untuk Deployment**

## **Konfigurasi Autentikasi Ngrok dan Menjalankan Aplikasi Streamlit**
Cell ini digunakan untuk mengatur *authentication token Ngrok* dan menjalankan aplikasi Streamlit secara lokal. Token diperlukan agar *Ngrok* dapat membuat tunnel dengan akun pengguna. Setelah token diatur, aplikasi Streamlit dijalankan menggunakan subprocess sehingga server lokal aktif di background.

Apabila Anda belum mengetahui cara mendapatkan *authentication token Ngrok* milik Anda sendiri, silahkan baca pada bagian **tips and tricks submission**.

In [8]:
auth_token = "39GpiEmLfTsqzwUo1PeDo4voTDV_7Et2S21SA73jQJMAsAuEJ"

ngrok.set_auth_token(auth_token)
subprocess.Popen(["streamlit", "run", "app.py"])

<Popen: returncode: None args: ['streamlit', 'run', 'app.py']>

## **Membuat Public URL**
Cell ini membuat *tunnel Ngrok* ke port lokal tempat Streamlit berjalan (default: 8501). *Ngrok* kemudian menghasilkan public URL yang bisa diakses dari internet, sehingga aplikasi Streamlit dapat dibuka tanpa harus berada di jaringan lokal yang sama.

In [9]:
public_url = ngrok.connect(8501).public_url
print(public_url)

https://nontroubling-nonproducible-josiah.ngrok-free.dev


Apabila Anda mengalami limit endpoint usage pada Ngrok, jalankan hidden cell di bawah ini!

## **Menutup Semua Tunnel Ngrok yang Aktif**
Cell ini digunakan untuk menghentikan seluruh koneksi Ngrok yang masih aktif.

In [10]:
ngrok.kill()
print("All active ngrok tunnels have been closed.")

All active ngrok tunnels have been closed.
